In [1]:
#import libraries
import numpy as np 
import matplotlib as plt
import tensorflow as tf 
from tensorflow.keras import layers ,models ,Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator ,load_img 
from tensorflow.keras.applications.resnet50 import preprocess_input , decode_predictions,ResNet50
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,BatchNormalization,GlobalAveragePooling2D

In [2]:
#Define training and test files
img_height, img_width = (64,64)
batch_size= 32
train_data = f"data/train"
test_data = f"data/test"

In [3]:
class_mode="categorical"

In [4]:
#Divide into training, testing and validation
train_data_1 = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  validation_split=0.4)
train_genter = train_data_1.flow_from_directory(train_data,
                                               target_size=(img_height,img_width),
                                               batch_size=batch_size,
                                               class_mode="categorical")

Found 3010 images belonging to 7 classes.


In [5]:
test_genter = train_data_1.flow_from_directory(test_data,
                                               target_size=(img_height,img_width),
                                               batch_size=batch_size,
                                               class_mode="categorical")

Found 770 images belonging to 7 classes.


In [6]:
x,y = test_genter.next()
x.shape

(32, 64, 64, 3)

### Create more than one neural network and train it on images

In [7]:
#
model_cs = ResNet50(include_top=False,weights="imagenet")
x = model_cs.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation="relu")(x)
pred = Dense(train_genter.num_classes,activation='softmax')(x)
model = Model(inputs=model_cs.input,outputs=pred)
for layer in model_cs.layers:
    layer.trainable = False
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_genter, epochs=20)

Epoch 1/20
95/95 [==============================] - 21s 188ms/step - loss: 2.8501 - accuracy: 0.2615
Epoch 2/20
95/95 [==============================] - 18s 186ms/step - loss: 1.6661 - accuracy: 0.3688
Epoch 3/20
95/95 [==============================] - 18s 193ms/step - loss: 1.5796 - accuracy: 0.4063
Epoch 4/20
95/95 [==============================] - 19s 195ms/step - loss: 1.5348 - accuracy: 0.4159
Epoch 5/20
95/95 [==============================] - 19s 194ms/step - loss: 1.4500 - accuracy: 0.4678
Epoch 6/20
95/95 [==============================] - 19s 195ms/step - loss: 1.4178 - accuracy: 0.4757
Epoch 7/20
95/95 [==============================] - 19s 202ms/step - loss: 1.3484 - accuracy: 0.4927
Epoch 8/20
95/95 [==============================] - 19s 195ms/step - loss: 1.3120 - accuracy: 0.5110
Epoch 9/20
95/95 [==============================] - 19s 196ms/step - loss: 1.2433 - accuracy: 0.5415
Epoch 10/20
95/95 [==============================] - 19s 195ms/step - loss: 1.2215 - accura

In [8]:
model.evaluate(test_genter)

25/25 [==============================] - 6s 176ms/step - loss: 1.9409 - accuracy: 0.3922


[1.9408588409423828, 0.3922078013420105]

In [9]:
cnn = Sequential([
                  layers.Conv2D(32, (2,2), activation='relu', input_shape=(64, 64, 3)),
                  layers.MaxPooling2D(3,3),
                  layers.Conv2D(64, kernel_size=(3,3),activation='relu',kernel_initializer='he_normal'),
                  layers.MaxPooling2D(2,2),
                  #layers.Conv2D(64, (2, 2), activation='relu'),
                  layers.Conv2D(64, kernel_size=(2,2),activation='relu',kernel_initializer='he_normal'),
                  layers.MaxPooling2D(2,2),
                  layers.Flatten(),
                  layers.Dense(128,activation='relu'),
                  layers.Dense(7,activation='softmax',kernel_initializer='glorot_normal')
                  
])

In [10]:
cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
cnn.fit(train_genter, epochs=30)

Epoch 1/30
95/95 [==============================] - 8s 75ms/step - loss: 6.7860 - accuracy: 0.1635
Epoch 2/30
95/95 [==============================] - 7s 71ms/step - loss: 1.9968 - accuracy: 0.1781
Epoch 3/30
95/95 [==============================] - 7s 74ms/step - loss: 1.9575 - accuracy: 0.1671
Epoch 4/30
95/95 [==============================] - 7s 73ms/step - loss: 1.9382 - accuracy: 0.1701
Epoch 5/30
95/95 [==============================] - 7s 70ms/step - loss: 1.9362 - accuracy: 0.1691
Epoch 6/30
95/95 [==============================] - 7s 70ms/step - loss: 1.9438 - accuracy: 0.1561
Epoch 7/30
95/95 [==============================] - 7s 71ms/step - loss: 1.9378 - accuracy: 0.1641
Epoch 8/30
95/95 [==============================] - 7s 71ms/step - loss: 1.9182 - accuracy: 0.2013
Epoch 9/30
95/95 [==============================] - 7s 71ms/step - loss: 1.8856 - accuracy: 0.2239
Epoch 10/30
95/95 [==============================] - 7s 70ms/step - loss: 1.8724 - accuracy: 0.2309
Epoch 11/

In [11]:
cnn.evaluate(test_genter)

25/25 [==============================] - 1s 51ms/step - loss: 1.7170 - accuracy: 0.3442


[1.7170146703720093, 0.34415584802627563]

In [12]:
#We save the neural network with the highest score
model.save("model.h5")